In [1]:
%matplotlib ipympl

In [2]:
import sys
import matplotlib.pyplot as plt
sys.path.append('/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Code/')


In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.nn.pool import max_pool_x

import importlib

In [4]:
import Dataset2 as Dataset

importlib.reload(Dataset)
DatasetContainer = Dataset.DatasetContainer
ProcessingDatasetContainer = Dataset.ProcessingDatasetContainer

In [5]:
ProcDS = torch.load('/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Data/Axis_Conv3d_Dataset.pt')
from DataGenFunctions import Unnormalise_Axis_Xmax_Energy


ProcDS.Unnormalise_Truth = Unnormalise_Axis_Xmax_Energy

torch.save(ProcDS, '/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Data/Axis_Conv3d_Dataset.pt')

In [6]:
if 'ProcDS' not in globals():
    ProcDS = torch.load('/remote/tychodata/ftairli/work/Projects/ProfileReconstruction/Data/Axis_Conv3d_Dataset.pt')
    ProcDS.AssignIndices()

ProcDS.BatchSize = 64
ProcDS.RandomIter = True

In [96]:
import Model_Geom_Graph as Models
importlib.reload(Models)
Model_Geom_Graph_C = Models.Model_Geom_Graph_C
model = Model_Geom_Graph_C()

loss = Models.Loss


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.to('cuda')
model.train()


Model_Geom_Graph_C(
  (Graph1): GATConv(5, 5, heads=16)
  (Graph2): GATConv(80, 5, heads=16)
  (Graph3): GATConv(80, 5, heads=16)
  (GraphS): GATConv(80, 5, heads=16)
  (Conv1): Conv1d(160, 16, kernel_size=(10,), stride=(2,), padding=(1,))
  (Conv2): Conv1d(16, 16, kernel_size=(10,), stride=(2,), padding=(1,))
  (Conv3): Conv1d(16, 16, kernel_size=(10,), stride=(2,), padding=(1,))
  (ConvActivation): LeakyReLU(negative_slope=0.01)
  (AvgPool): AvgPool1d(kernel_size=(10,), stride=(10,), padding=(0,))
  (MaxPool): MaxPool1d(kernel_size=10, stride=10, padding=0, dilation=1, ceil_mode=False)
  (Dense1): Linear(in_features=352, out_features=128, bias=True)
  (Dense2): Linear(in_features=128, out_features=128, bias=True)
  (Dense3): Linear(in_features=128, out_features=128, bias=True)
  (Chi01): Linear(in_features=128, out_features=128, bias=True)
  (Chi02): Linear(in_features=128, out_features=128, bias=True)
  (Chi03): Linear(in_features=128, out_features=1, bias=True)
  (Rp1): Linear(in_f

In [97]:
# train for a bit
Batches_to_compute = 1000
ProcDS.BatchSize = 40

for i,(_,BatchGraph, BatchAux, BatchTruth, _) in enumerate(ProcDS):
    print(i, end='\r')
    if i == Batches_to_compute: break
    optimizer.zero_grad()

    preds = model(BatchGraph, BatchAux)
    losses = loss(preds, BatchTruth)
    totalLoss = losses['Total']
    totalLoss.backward()
    optimizer.step()
print()
print(totalLoss)




1000
tensor(1.7970, device='cuda:0', grad_fn=<AddBackward0>)


In [102]:
# Step by step training
Batches_to_compute = 1
ProcDS.BatchSize = 1
for i,(EvId,BatchGraph, BatchAux, BatchTruth, _) in enumerate(ProcDS):
    print(EvId.item())
    print()
    optimizer.zero_grad()
    preds = model(BatchGraph, BatchAux,True)
    break
    


1077301248.0

tensor([[  4.5166, -12.5192, -27.7353,  ...,  33.0060,  87.0572,  -5.1908],
        [  4.7468, -12.5463, -27.6503,  ...,  33.0952,  86.7947,  -5.1267],
        [  4.6157, -12.5388, -27.7070,  ...,  33.0326,  86.9485,  -5.1822],
        ...,
        [ 37.9396, -25.8989, -32.4871,  ...,  51.0374,  84.8433,  -9.9767],
        [ 37.9389, -25.8821, -32.4613,  ...,  51.0441,  84.8156,  -9.9456],
        [ 38.0024, -25.9121, -32.4803,  ...,  51.0623,  84.8045,  -9.9721]],
       device='cuda:0', grad_fn=<IndexBackward0>)
tensor([  0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,
          0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000,   0.0000

/remote/tychodata/ftairli/.local/lib/python3.9/site-packages/torch_geometric/warnings.py:17: UserWarning: The usage of `scatter(reduce='max')` can be accelerated via the 'torch-scatter' package, but it was not found
  warnings.warn(message)
